# 🔐 Divine Convergence – Robust D1 Chart Parser (All Planets + Firebase Upload)

## Step 0: Upload Firebase Key

In [ ]:

from google.colab import files
uploaded = files.upload()

import json
import os

for fn in uploaded.keys():
    if fn.endswith('.json'):
        firebase_key_path = fn
        break


## Step 1: Install Firebase SDK

In [ ]:
!pip install firebase-admin

## Step 2: Initialize Firebase

In [ ]:

import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate(firebase_key_path)
firebase_admin.initialize_app(cred)
db = firestore.client()


## Step 3: Parsing Utilities (All Points)

In [ ]:

import re
import urllib.parse

zodiac_signs = [
    "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
    "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
]

def degree_to_sign(degree_float):
    sign_index = int(degree_float // 30)
    degree_within_sign = round(degree_float % 30, 2)
    return f"{degree_within_sign:.2f}° {zodiac_signs[sign_index]}"

def build_astroseek_chart_url(birth_data):
    base_url = "https://horoscopes.astro-seek.com/birth-chart-horoscope-online"
    params = {
        "narozeni_den": birth_data["day"],
        "narozeni_mesic": birth_data["month"],
        "narozeni_rok": birth_data["year"],
        "narozeni_hodina": birth_data["hour"],
        "narozeni_minuta": birth_data["minute"],
        "narozeni_mesto": birth_data["city"],
        "send": "Send"
    }
    return f"{base_url}?{urllib.parse.urlencode(params)}"

def parse_astroseek_d1(url):
    pattern = re.compile(r"p_([a-zA-Z_]+)=([\d.]+)")
    matches = pattern.findall(url)

    aliases = {
        "slunce": "Sun", "sun": "Sun",
        "luna": "Moon", "moon": "Moon",
        "mercury": "Mercury", "venus": "Venus", "mars": "Mars",
        "jupiter": "Jupiter", "saturn": "Saturn",
        "uran": "Uranus", "neptun": "Neptune", "pluto": "Pluto",
        "uzel": "North Node", "lilith": "Lilith", "chiron": "Chiron",
        "asc": "Ascendant", "vertex": "Vertex", "fortune": "Fortune", "spirit": "Spirit"
    }

    chart_data = {}
    for raw_planet, value in matches:
        try:
            degree = float(value)
            key = raw_planet.lower()
            planet_name = aliases.get(key, key.capitalize())
            chart_data[planet_name] = degree_to_sign(degree)
        except ValueError:
            print(f"⚠️ Skipping invalid value for {raw_planet}: {value}")
    return chart_data


## Step 4: Push to Firebase

In [ ]:

def push_to_firebase(user_id, birth_data, chart_data):
    user_ref = db.collection('users').document(user_id)
    user_ref.set({
        'birthData': birth_data,
        'chartData': chart_data,
        'matchStatus': 'waiting'
    })
    print(f"Pushed {user_id} to Firebase.")


## Step 5: Example Usage

In [ ]:

birth_data = {
    "day": 2,
    "month": 7,
    "year": 1985,
    "hour": 5,
    "minute": 25,
    "city": "Baltimore"
}

astroseek_url = build_astroseek_chart_url(birth_data)
print("Generated Astro-Seek URL:", astroseek_url)


In [ ]:

# ⚠️ Manual Step: Paste full chart URL with planetary values
pdf_chart_url = "https://horoscopes.astro-seek.com/...&p_sun=100.437&p_moon=278.938&p_mercury=123.89..."

parsed_chart = parse_astroseek_d1(pdf_chart_url)
print("\nParsed D1 Chart:")
for planet, position in parsed_chart.items():
    print(f"{planet}: {position}")

push_to_firebase("user_test_19850702", birth_data, parsed_chart)
